In [1]:
import os
import sys
import random
import warnings

from keras.utils import to_categorical
from keras import activations
from keras import backend as K
from keras.engine.topology import Layer
from keras import optimizers
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model

from keras import utils
from keras.datasets import mnist
from keras.models import Model
from keras.layers import *


from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
session = tf.Session(config=config)

KTF.set_session(session )

/home/c1501f/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def normalization_processing(data):
    
    data_mean = data.mean()
    data_var = data.var()
    
    data = data - data_mean
    data = data / data_var
    
    return data

In [3]:
train_batch = 196
test_batch = 196

train_num = 142420
test_num = 45668

cross = "cross_1"

snr_str = "-6_detail_db"

In [4]:
def generate_1d_train_data():
    
    train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)

    
    train_lab = np.zeros((train_batch))
    flag = 0
    
    while True:
        
        list = random.sample(range(train_num), train_num)
        
        for id in list:
            
            num_id = str(id)
            
            line_train_data = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/train_data/" + num_id + "_train.npy")
            line_train_lab = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/train_lab/" + num_id + "_lab.npy")
            
            line_train_data = normalization_processing(line_train_data)

            train_data[flag , :, 0] = line_train_data

            train_lab[flag] = line_train_lab
            
            flag += 1
            
            if flag >= train_batch:
                
                train_hot_lab = to_categorical(train_lab, num_classes=12)
                
                yield [train_data], [train_hot_lab]
                
                flag = 0
                train_data = np.zeros((train_batch, 2048, 1), dtype = np.float32)
                
                train_lab = np.zeros((train_batch))

        
def generate_1d_test_data():
    
    test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
    
    test_lab = np.zeros((test_batch))
    flag = 0
    
    while True:
        
        list = random.sample(range(test_num), test_num)
        
        for id in list:
            
            num_id = str(id)
            
            line_test_data = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
            line_test_lab = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
            
            line_test_data = normalization_processing(line_test_data)
            
            test_data[flag , :, 0] = line_test_data
            
            test_lab[flag] = line_test_lab
            
            flag += 1
            
            if flag >= train_batch:
                
                test_hot_lab = to_categorical(test_lab, num_classes=12)
                
                yield [test_data], [test_hot_lab]
                
                flag = 0
                test_data = np.zeros((test_batch, 2048, 1), dtype = np.float32)
          
                test_lab = np.zeros((test_batch))

In [5]:
def get_all_test_data():
    
    test_data = np.zeros((test_num, 2048, 1), dtype = np.float32)
    
    test_lab = np.zeros((test_num))
    
    flag = 0
    list = random.sample(range(test_num), test_num)
    
    for id in list:
        
        num_id = str(id)
        
        line_test_data = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_data/" + num_id + "_test.npy")
        line_test_lab = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_lab/" + num_id + "_lab.npy")
        
        line_test_data = normalization_processing(line_test_data)

        test_data[flag , :, 0] = line_test_data
        
        test_lab[flag] = line_test_lab
        
        flag += 1
        
    test_hot_lab = to_categorical(test_lab, num_classes=12)
    
    return test_data, test_hot_lab

In [6]:
def conv1D_bn(x, filters, kernel_size, strides):
    
    C1 = Conv1D(filters, kernel_size, strides=strides, padding='same')(x) 
    C1 = Activation('relu')(C1)
    
    return C1

In [7]:
def A_CAM(in_x, nb_filter):

    c = GlobalAveragePooling1D()(in_x)
    c = Reshape((1, nb_filter))(c)
    
    c = Conv1D(int(nb_filter/2), 1, padding='same')(c) 
    #c = BatchNormalization()(c)
    c = Activation('relu')(c)
    
    c = Conv1D(nb_filter, 1, padding='same')(c)
    c = BatchNormalization()(c)
    c = Activation('sigmoid')(c)
    
    x = multiply([in_x, c])
    out = add([in_x, x])
    
    return out


def A_EAM(in_x, nb_filter, kernel_size):
        
    t = Conv1D(1, 1, padding='same')(in_x)
    t = BatchNormalization()(t)
    t = Activation('sigmoid')(t)
    
    x = Conv1D(nb_filter, kernel_size, padding='same')(in_x)
    x = Activation('relu')(x)
    
    x = multiply([x, t])
    out = add([in_x, x])
    
    return out


def A_JAM(in_x, nb_filter, kernel_size):
    
    cx = A_CAM(in_x, nb_filter)
    tx = A_tSE(cx, nb_filter, kernel_size)
    
    return tx   

In [8]:
def MA1DCNN():
    
    input_signal = Input(shape=(2048,1))

    cnn = Conv1D(32, kernel_size = 32, activation='relu', padding='same', strides=1)(input_signal)
    cnn = A_EAM(cnn, 32, 32)
    cnn = A_CAM(cnn, 32)

    cnn = Conv1D(32, kernel_size = 16, activation='relu', padding='same', strides=2)(cnn)   
    cnn = A_EAM(cnn, 32, 16)
    cnn = A_CAM(cnn, 32)

    cnn = Conv1D(64, kernel_size = 9, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_EAM(cnn, 64, 9)
    cnn = A_CAM(cnn, 64)    

    cnn = Conv1D(64, kernel_size = 6, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_EAM(cnn, 64, 6)
    cnn = A_CAM(cnn, 64)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    cnn = A_EAM(cnn, 128, 3)
    cnn = A_CAM(cnn, 128)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    
    x = GlobalAveragePooling1D()(cnn)
    
    output = Dense(12, activation='softmax')(x)

    model = Model(inputs=input_signal, outputs=output)
    
    return model

In [10]:
def ACAM_CNN():
    
    input_signal = Input(shape=(2048,1))

    cnn = Conv1D(32, kernel_size = 32, activation='relu', padding='same', strides=1)(input_signal)
    cnn = A_CAM(cnn, 32)

    cnn = Conv1D(32, kernel_size = 16, activation='relu', padding='same', strides=2)(cnn)   
    cnn = A_CAM(cnn, 32)

    cnn = Conv1D(64, kernel_size = 9, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_CAM(cnn, 64)

    cnn = Conv1D(64, kernel_size = 6, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_CAM(cnn, 64)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    cnn = A_CAM(cnn, 128)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    
    x = GlobalAveragePooling1D()(cnn)
    
    output = Dense(12, activation='softmax')(x)

    model = Model(inputs=input_signal, outputs=output)
    
    return model


def AEAM_CNN():
    
    input_signal = Input(shape=(2048,1))

    cnn = Conv1D(32, kernel_size = 32, activation='relu', padding='same', strides=1)(input_signal)
    cnn = A_EAM(cnn, 32, 32)

    cnn = Conv1D(32, kernel_size = 16, activation='relu', padding='same', strides=2)(cnn)   
    cnn = A_EAM(cnn, 32, 16)

    cnn = Conv1D(64, kernel_size = 9, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_EAM(cnn, 64, 9)

    cnn = Conv1D(64, kernel_size = 6, activation='relu', padding='same', strides=2)(cnn)
    cnn = A_EAM(cnn, 64, 6)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    cnn = A_EAM(cnn, 128, 3)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)

    
    x = GlobalAveragePooling1D()(cnn)
    
    output = Dense(12, activation='softmax')(x)

    model = Model(inputs=input_signal, outputs=output)
    
    return model

In [11]:
def A_CNN():
    
    input_signal = Input(shape=(2048,1))

    cnn = Conv1D(32, kernel_size = 32, activation='relu', padding='same', strides=1)(input_signal)

    cnn = Conv1D(32, kernel_size = 16, activation='relu', padding='same', strides=2)(cnn)   

    cnn = Conv1D(64, kernel_size = 9, activation='relu', padding='same', strides=2)(cnn)

    cnn = Conv1D(64, kernel_size = 6, activation='relu', padding='same', strides=2)(cnn)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    
    cnn = Conv1D(128, kernel_size = 3, activation='relu', padding='same', strides=4)(cnn)
    
    x = GlobalAveragePooling1D()(cnn)
    
    output = Dense(12, activation='softmax')(x)

    model = Model(inputs=input_signal, outputs=output)
    
    return model

In [12]:
def AJAM_WDCNN():
    
    input_signal = Input((2048, 1))
    
    x = Conv1D(16, 64, strides=16, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = A_EAM(x, 16, 64)
    x = A_CAM(x, 16)   
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(32, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = A_EAM(x, 32, 3)
    x = A_CAM(x, 32)      
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(64, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = A_EAM(x, 64, 3)
    x = A_CAM(x, 64)      
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(64, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = A_EAM(x, 64, 3)
    x = A_CAM(x, 64)    
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(64, 3, padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling1D(2)(x)
    
    x = Flatten()(x)
    
    x = Dense(100,  activation='relu')(x)
    
    x = Dense(12,  activation='softmax')(x)
    
    model = Model(input_signal, x)
    
    return model

In [13]:
adam = optimizers.adam(lr = 0.0001)

model = MA1DCNN()

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ["accuracy"])

model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2048, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 2048, 32)     1056        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 2048, 1)      33          conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 2048, 32)     32800       conv1d_1[0][0]                   
__________________________

In [14]:
class MyCbk(Callback):

    def __init__(self, model):
        self.model_to_save = model
            
    def on_epoch_end(self, epoch, logs=None):
        
        print('save model----model_at_epoch_%d.h5' % epoch)
        self.model_to_save.save('model_1/model_%d.h5' % epoch)
        
cbk = MyCbk(model)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.90, patience=5, verbose=0, mode='auto', cooldown=0, min_lr=0.0000001)

results = model.fit_generator(generate_1d_train_data(), steps_per_epoch = train_num/train_batch, epochs = 100, validation_data = generate_1d_test_data(), validation_steps = test_num/test_batch, verbose=1, callbacks=[cbk, reduceLR])


Epoch 1/100
727/726 [==============================] - 234s 322ms/step - loss: 1.4719 - acc: 0.4545 - val_loss: 1.2517 - val_acc: 0.5176
save model----model_at_epoch_0.h5
Epoch 2/100
727/726 [==============================] - 211s 290ms/step - loss: 1.1115 - acc: 0.5782 - val_loss: 1.0882 - val_acc: 0.6045
save model----model_at_epoch_1.h5
Epoch 3/100
727/726 [==============================] - 204s 281ms/step - loss: 0.8064 - acc: 0.6995 - val_loss: 0.7498 - val_acc: 0.7236
save model----model_at_epoch_2.h5
Epoch 4/100
727/726 [==============================] - 200s 276ms/step - loss: 0.5653 - acc: 0.7905 - val_loss: 0.6194 - val_acc: 0.7749
save model----model_at_epoch_3.h5
Epoch 5/100
727/726 [==============================] - 200s 275ms/step - loss: 0.4281 - acc: 0.8424 - val_loss: 0.9439 - val_acc: 0.7443
save model----model_at_epoch_4.h5
Epoch 6/100
727/726 [==============================] - 199s 274ms/step - loss: 0.3370 - acc: 0.8763 - val_loss: 0.6063 - val_acc: 0.7877
save mod

KeyboardInterrupt: 

In [15]:
all_test_data, all_test_lab = get_all_test_data()


Accuracy_list = np.zeros((100))
Recall_list = np.zeros((100))
Precision_list = np.zeros((100))

In [16]:
for model_id in range(100):
    
    str_id = str(model_id)
    
    model =  load_model('model_1/model_'+ str_id +'.h5') 
    
    all_score = 0
    
    range_num = int(test_num/test_batch)
    
    input_lab = []
    output_lab = []
    
    for num_id in range(range_num):
        
        test_data = all_test_data[num_id*test_batch:test_batch+num_id*test_batch,:,:]
        
        test_lab = all_test_lab[num_id*test_batch:test_batch+num_id*test_batch,:]
    
        pre_lab = model.predict(test_data, batch_size=test_batch, verbose=0)
             
        input_lab.extend(test_lab)
        output_lab.extend(pre_lab)

    labels_test = np.argmax(input_lab,axis=1)  
    labels_pred = np.argmax(output_lab,axis=1)
          
    Accuracy = accuracy_score(labels_test, labels_pred)
    Recall = recall_score(labels_test, labels_pred, average='macro')
    #F1_score = f1_score(labels_test, labels_pred, average='macro')
    Precision = precision_score(labels_test, labels_pred, average='macro')
    
    Accuracy_list[model_id] = Accuracy
    Recall_list[model_id] = Recall
    Precision_list[model_id] = Precision
    
    print("第"+ str(model_id) + "个模型的结果：")
    print(Accuracy)
    print(Recall)
    print(Precision)
    
    print("------------------------------------")
    
    K.clear_session()
    tf.reset_default_graph()


/home/c1501f/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


第0个模型的结果：
0.5175615310501883
0.43814934298033786
0.42098025842361114
------------------------------------
第1个模型的结果：
0.6045151966365946
0.5251392640247031
0.5384809748004432
------------------------------------
第2个模型的结果：
0.7236358062538321
0.6767394539934148
0.7109982213431106
------------------------------------
第3个模型的结果：
0.774853288955067
0.7409510101365298
0.7548192288670988
------------------------------------
第4个模型的结果：
0.7442629412279933
0.6935714803170833
0.7585717440728352
------------------------------------
第5个模型的结果：
0.7877069282648682
0.7938955901529748
0.7950809456765522
------------------------------------
第6个模型的结果：
0.8133704125426995
0.7800017260929586
0.8054403729826465
------------------------------------
第7个模型的结果：
0.7760138390119996
0.7438355877970539
0.8034970307351433
------------------------------------
第8个模型的结果：
0.8071735131820968
0.8106748278363142
0.81780615042158
------------------------------------
第9个模型的结果：
0.8217570289918542
0.7994735662691824
0.819760295868429

第79个模型的结果：
0.8184067618463694
0.7956758957665241
0.8198714625490154
------------------------------------
第80个模型的结果：
0.8252167819917666
0.8056562658385295
0.8163994916827827
------------------------------------
第81个模型的结果：
0.8271437330296926
0.8069502414108348
0.8208989582300469
------------------------------------
第82个模型的结果：
0.8191512656564772
0.8061556327978522
0.8105817308999629
------------------------------------
第83个模型的结果：
0.8174870806691775
0.7991521317078513
0.8128537217607646
------------------------------------
第84个模型的结果：
0.8182315844792852
0.7949444644807494
0.8132978263431442
------------------------------------
第85个模型的结果：
0.8288298151878777
0.809440836052846
0.8247287112988518
------------------------------------
第86个模型的结果：
0.804283086625208
0.8011138110879653
0.8017301256295345
------------------------------------
第87个模型的结果：
0.8207935534728913
0.8016987605971226
0.8149948253343205
------------------------------------
第88个模型的结果：
0.8257204169221337
0.8051645731391971
0.817781

In [17]:
Accuracy_str = "["
Recall_str = "["
Precision_str = "["

for id in range(100):
    
    Accuracy_num = round(Accuracy_list[id], 5)
    Recall_num = round(Recall_list[id], 5)
    Precision_num = round(Precision_list[id], 5)

    Accuracy_str = Accuracy_str + str(Accuracy_num)
    Recall_str = Recall_str + str(Recall_num)
    Precision_str = Precision_str + str(Precision_num)
    
    if id == 99:
        Accuracy_str = Accuracy_str + "]"
        Recall_str = Recall_str + "]"
        Precision_str = Precision_str + "]"
    else:
        Accuracy_str = Accuracy_str + ", "
        Recall_str = Recall_str + ", "
        Precision_str = Precision_str + ", "

print("Accuracy_score:")
print(Accuracy_str)
print("Recall_score:")
print(Recall_str)
print("Precision_score:")
print(Precision_str)

Accuracy_score:
[0.51756, 0.60452, 0.72364, 0.77485, 0.74426, 0.78771, 0.81337, 0.77601, 0.80717, 0.82176, 0.81103, 0.78904, 0.80238, 0.80669, 0.79577, 0.77739, 0.78009, 0.80095, 0.81959, 0.8163, 0.77691, 0.80682, 0.81335, 0.81193, 0.74356, 0.82373, 0.76261, 0.79866, 0.82511, 0.82465, 0.80689, 0.82395, 0.82419, 0.79844, 0.80643, 0.80187, 0.79136, 0.77507, 0.80665, 0.7678, 0.78539, 0.81668, 0.77993, 0.82066, 0.82268, 0.80649, 0.81418, 0.79504, 0.82463, 0.7941, 0.81525, 0.78718, 0.81659, 0.80702, 0.81363, 0.81223, 0.81992, 0.82723, 0.77376, 0.8137, 0.82835, 0.82795, 0.8098, 0.82062, 0.79633, 0.81738, 0.80349, 0.81442, 0.8103, 0.80639, 0.71378, 0.81052, 0.81076, 0.8223, 0.83049, 0.82763, 0.83078, 0.83209, 0.83225, 0.81841, 0.82522, 0.82714, 0.81915, 0.81749, 0.81823, 0.82883, 0.80428, 0.82079, 0.82572, 0.81955, 0.82574, 0.82417, 0.82636, 0.82992, 0.82605, 0.79771, 0.82206, 0.82099, 0.81503, 0.82776]
Recall_score:
[0.43815, 0.52514, 0.67674, 0.74095, 0.69357, 0.7939, 0.78, 0.74384, 0.81067

(45668,) [0 5 0 ... 8 5 1]


In [ ]:
'''
test_num = 11172
test_batch = 133
class_num = "0"

test_num = 3724
test_batch = 133
class_num = "1"

test_num = 4704
test_batch = 147
class_num = "2"

test_num = 2058
test_batch = 147
class_num = "3"

test_num = 1960
test_batch = 196
class_num = "4"

test_num = 3920
test_batch = 245
class_num = "5"

test_num = 3430
test_batch = 343
class_num = "6"

test_num = 3136
test_batch = 196
class_num = "7"

test_num = 3234
test_batch = 231
class_num = "8"

test_num = 3528
test_batch = 147
class_num = "9"

test_num = 3920
test_batch = 245
class_num = "10"

test_num = 882
test_batch = 147
class_num = "11"
'''

In [38]:
test_num = 2058
test_batch = 147
class_num = "3"


def get_0_test_data():
    
    test_data = np.zeros((test_num, 2048, 1), dtype = np.float32)
    
    test_lab = np.zeros((test_num))
    
    flag = 0
    list = random.sample(range(test_num), test_num)
    
    for id in list:
        
        num_id = str(id)
        
        line_test_data = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_detail/"+class_num+"/" + num_id + "_test.npy")
        line_test_lab = np.load("../../H-S_data/noise_data/"+cross+"/"+snr_str+"/test_lab_detail/"+class_num+"/" + num_id + "_lab.npy")
        
        line_test_data = normalization_processing(line_test_data)

        test_data[flag , :, 0] = line_test_data
        
        test_lab[flag] = line_test_lab
        
        flag += 1
        
    test_hot_lab = to_categorical(test_lab, num_classes=12)
    
    return test_data, test_hot_lab

In [39]:
all_test_data, all_test_lab = get_0_test_data()


In [40]:
model =  load_model('model_1/model_94.h5') 

all_score = 0

range_num = int(test_num/test_batch)

input_lab = []
output_lab = []

for num_id in range(range_num):

    test_data = all_test_data[num_id*test_batch:test_batch+num_id*test_batch,:,:]

    test_lab = all_test_lab[num_id*test_batch:test_batch+num_id*test_batch,:]

    pre_lab = model.predict(test_data, batch_size=test_batch, verbose=0)

    input_lab.extend(test_lab)
    output_lab.extend(pre_lab)

labels_test = np.argmax(input_lab,axis=1)  
labels_pred = np.argmax(output_lab,axis=1)


print(labels_test.shape)

count_0 = np.sum(labels_pred == 0)
count_1 = np.sum(labels_pred == 1)
count_2 = np.sum(labels_pred == 2)
count_3 = np.sum(labels_pred == 3)
count_4 = np.sum(labels_pred == 4)
count_5 = np.sum(labels_pred == 5)
count_6 = np.sum(labels_pred == 6)
count_7 = np.sum(labels_pred == 7)
count_8 = np.sum(labels_pred == 8)
count_9 = np.sum(labels_pred == 9)
count_10 = np.sum(labels_pred == 10)
count_11 = np.sum(labels_pred == 11)

count_all_pre = count_0+count_1+count_2+count_3+count_4+count_5+count_6+count_7+count_8+count_9+count_10+count_11

print("pre-c0: "+str(count_0)+", 百分比："+str(count_0/count_all_pre*100))
print("pre-c1: "+str(count_1)+", 百分比："+str(count_1/count_all_pre*100))
print("pre-c2: "+str(count_2)+", 百分比："+str(count_2/count_all_pre*100))
print("pre-c3: "+str(count_3)+", 百分比："+str(count_3/count_all_pre*100))
print("pre-c4: "+str(count_4)+", 百分比："+str(count_4/count_all_pre*100))
print("pre-c5: "+str(count_5)+", 百分比："+str(count_5/count_all_pre*100))
print("pre-c6: "+str(count_6)+", 百分比："+str(count_6/count_all_pre*100))
print("pre-c7: "+str(count_7)+", 百分比："+str(count_7/count_all_pre*100))
print("pre-c8: "+str(count_8)+", 百分比："+str(count_8/count_all_pre*100))
print("pre-c9: "+str(count_9)+", 百分比："+str(count_9/count_all_pre*100))
print("pre-c10: "+str(count_10)+", 百分比："+str(count_10/count_all_pre*100))
print("pre-c11: "+str(count_11)+", 百分比："+str(count_11/count_all_pre*100))
print("Pre-all: "+str(count_all_pre))



Accuracy = accuracy_score(labels_test, labels_pred)


Accuracy_list[model_id] = Accuracy


print("第"+ str(model_id) + "个模型的结果：")
print(Accuracy)


print("------------------------------------")

K.clear_session()
tf.reset_default_graph()


(2058,)
pre-c0: 4, 百分比：0.1943634596695821
pre-c1: 51, 百分比：2.478134110787172
pre-c2: 296, 百分比：14.382896015549077
pre-c3: 1405, 百分比：68.27016520894071
pre-c4: 1, 百分比：0.048590864917395525
pre-c5: 44, 百分比：2.1379980563654035
pre-c6: 27, 百分比：1.3119533527696794
pre-c7: 72, 百分比：3.498542274052478
pre-c8: 37, 百分比：1.7978620019436347
pre-c9: 27, 百分比：1.3119533527696794
pre-c10: 94, 百分比：4.56754130223518
pre-c11: 0, 百分比：0.0
Pre-all: 2058
第99个模型的结果：
0.6827016520894071
------------------------------------
